<a href="https://colab.research.google.com/github/HalilHuseyinCaliskan/HalilHuseyinCaliskan/blob/main/Code_and_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics -U

In [ ]:
from google.colab import drive
drive.mount('/content/drive/MyDrive/kaynaklar')

In [ ]:
### Normal(Rakip) Tespit Modelinin Kodları

In [ ]:
from ultralytics import YOLO
import cv2, torch, time
cap = cv2.VideoCapture("/content/drive/MyDrive/kaynaklar/f22.mp4")
model = YOLO("/content/drive/MyDrive/kaynaklar/hızlı2.pt")
while True:
    ret, frame = cap.read()
    t = time.time()
    frame = cv2.resize(frame, (1920, 840))
    results = model(frame)[0]
    boxes = results.boxes
    for box in boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 1)
    torch.cuda.empty_cache()
    t2 = time.time()
    fps = 1 / (t2 - t)
    print(f"FPS: {fps:.2f}")
    listem=[]
    listem.append(fps)
    print(sum(listem)/len(listem))
    #cv2.imshow("frame", frame)
    if cv2.waitKey(1) & 0xFF == ord("s"):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
### Makine Öğrenmesi Destekli Tespit Modelimin Kodları

In [ ]:
import pandas as pd
import numpy as np
import cv2, time,torch
from ultralytics import YOLO

dataset = pd.read_csv("/content/drive/MyDrive/kaynaklar/listeler.csv")
dataset2 = pd.read_csv("/content/drive/MyDrive/kaynaklar/listeler2.csv")
dataset3 = pd.read_csv("/content/drive/MyDrive/kaynaklar/listeler3.csv")
dataset4 = pd.read_csv("/content/drive/MyDrive/kaynaklar/listeler4.csv")

X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values
X2 = dataset2.iloc[:, :-1].values
y2 = dataset2.iloc[:, -1].values
X3 = dataset3.iloc[:, :-1].values
y3 = dataset3.iloc[:, -1].values
X4 = dataset4.iloc[:, :-1].values
y4 = dataset4.iloc[:, -1].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=1)
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, test_size=0.2, random_state=1)
X_train4, X_test4, y_train4, y_test4 = train_test_split(X4, y4, test_size=0.2, random_state=1)

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import numpy as np

regressor = RandomForestRegressor(n_estimators=10, random_state=42)
regressor2 = RandomForestRegressor(n_estimators=10, random_state=42)
regressor3 = RandomForestRegressor(n_estimators=10, random_state=42)
regressor4 = RandomForestRegressor(n_estimators=10, random_state=42)

regressor.fit(X_train, y_train)
regressor2.fit(X_train2, y_train2)
regressor3.fit(X_train3, y_train3)
regressor4.fit(X_train4, y_train4)

y_pred = regressor.predict(X_test)
y_pred2 = regressor2.predict(X_test2)
y_pred3 = regressor3.predict(X_test3)
y_pred4 = regressor4.predict(X_test4)

CompareRandomForest = np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1)

R2ScoreRF = r2_score(y_test, y_pred)
R2ScoreRF2 = r2_score(y_test2, y_pred2)
R2ScoreRF3 = r2_score(y_test3, y_pred3)
R2ScoreRF4 = r2_score(y_test4, y_pred4)

print(R2ScoreRF, R2ScoreRF2, R2ScoreRF3, R2ScoreRF4)


class A:
    def __init__(self):
        self.result = None
        self.boxes = None

    def fonk(self):
        cap = cv2.VideoCapture("/content/drive/MyDrive/kaynaklar/f22.mp4")
        model = YOLO("/content/drive/MyDrive/kaynaklar/hızlı2.pt")
        liste, liste2, liste3, liste4 = [], [], [], []
        frame_count = 0
        min_history = 4

        while cap.isOpened():
            ret, frame = cap.read()
            t1 = time.time()
            frame = cv2.resize(frame, (1920, 840))
            if frame_count % 4 == 0:
                self.result = model(frame)[0]
                self.boxes = self.result.boxes
                for box in self.boxes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    liste.append(x1)
                    liste2.append(x2)
                    liste3.append(y1)
                    liste4.append(y2)
            else:
                if len(liste) >= min_history:

                        x1 = regressor.predict([[liste[-4], liste[-3], liste[-2], liste[-1]]])
                        x2 = regressor2.predict([[liste2[-4], liste2[-3], liste2[-2], liste2[-1]]])
                        y1 = regressor3.predict([[liste3[-4], liste3[-3], liste3[-2], liste3[-1]]])
                        y2 = regressor4.predict([[liste4[-4], liste4[-3], liste4[-2], liste4[-1]]])
                        x1, y1 = int(x1[0]), int(y1[0])
                        x2, y2 = int(x2[0]), int(y2[0])
                        liste.append(x1)
                        #liste.pop(0)
                        liste2.append(x2)
                        #liste2.pop(0)
                        liste3.append(y1)
                        #liste3.pop(0)
                        liste4.append(y2)
                        #liste4.pop(0)
                        print(x1,y1)
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 5)

            torch.cuda.empty_cache()
            frame_count += 1
            t2 = time.time()
            listem=[]
            listem.append(1 / (t2 - t1))
            print(sum(listem) / len(listem))
            #cv2.imshow("frame", frame)
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
        cap.release()
        cv2.destroyAllWindows()

nesne = A()
nesne.fonk()


In [ ]:
### Piksel Tabanlı Analiz Yapıp Görüntü Sınıflandıran Makine Öğrenmesi Modelim

In [ ]:
from ultralytics import YOLO
import cv2,time
import numpy as np
import csv
import pandas as pd
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings('ignore', category=ConvergenceWarning)
dataset = pd.read_csv("/content/drive/MyDrive/kaynaklar/merged_yeni.csv")
dataset=dataset[2:]
print(dataset.head)
X = dataset.iloc[:,:-1].values
y = dataset.iloc[:,-1].values
print(len(X[0]))
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.1,random_state=45)
print(X_train)
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)
from sklearn.decomposition import PCA
pca = PCA(n_components=128)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state=0,max_iter=1000)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
from sklearn.metrics import accuracy_score
AccuracyScore = accuracy_score(y_test,y_pred)
print(AccuracyScore)
class A:
    def __init__(self):
        self.alan=0
        self.max=0
        pass
    def fonk(self):
     cap = cv2.VideoCapture("/content/drive/MyDrive/kaynaklar/f22.mp4")
     model = YOLO("/content/drive/MyDrive/kaynaklar/hızlı2.pt")
     liste=[]
     liste2=[]
     while True:
      ret, frame = cap.read()
      frame=cv2.resize(frame,(640,640))
      results = model(frame)[0]
      boxes = results.boxes
      for box in boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        frame2=frame[y1:y2,x1:x2]
        frame3=frame2.copy()
        frame2=cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
        frame2 = cv2.bilateralFilter(frame2, 9, 75, 75)
        frame2=cv2.Canny(frame2,10,50)
        _,frame2=cv2.threshold(frame2,125,255,cv2.THRESH_BINARY)
        kernel = np.ones((3, 3), np.uint8)
        frame2 = cv2.dilate(frame2, kernel, iterations=1)
        frame2=cv2.resize(frame2,(16,16))
        liste=[]
        for i in range(len(frame2)):
            for j in range(len(frame2[0])):
                liste.append(frame2[i][j])
        listem = np.array(liste).reshape(1, -1)
        listem=pca.transform(listem)
        #print(listem)
        zaman=time.time()
        a=classifier.predict(listem)
        zaman2=time.time()
        print(zaman2-zaman)
fonksiyon=A()
fonksiyon.fonk()

In [ ]:
### Klasik Yapay Sinir Ağı Tabanlı Görüntü Sınıflandırma Modeli

In [ ]:
!pip install tensorflow
!pip install keras

In [ ]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import load_model
import cv2,time

model = load_model('/content/drive/MyDrive/kaynaklar/normal.h5')
cap = cv2.VideoCapture("/content/drive/MyDrive/kaynaklar/f22.mp4")
while True:

    ret, frame = cap.read()
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    resized_frame = cv2.resize(gray_frame, (16, 16))
    resized_frame_flattened = resized_frame.reshape(1, -1)
    start_time = time.time()
    prediction = model.predict(resized_frame_flattened)
    end_time = time.time()
    prediction_time = end_time - start_time
    print(prediction_time)

cap.release()
cv2.destroyAllWindows()